# Differential Evolution with Newton and custom initialisation

In [ ]:
import numpy as np
from math import isinf


In [ ]:

import numpy as np
from math import isinf

def fmin_DENewton(
    F, dF, ddF,
    lower_bounds, upper_bounds,
    beta_min, beta_max, pCR,
    n_pop, tol_cost, tol_grad, max_it,
    newton_LS_size=8,
    rng=None,
):
    rng = rng or np.random.default_rng()
    n_var = len(lower_bounds)
    lower_bounds = np.asarray(lower_bounds, dtype=float)
    upper_bounds = np.asarray(upper_bounds, dtype=float)

    # initialise population according to bounds
    pop_x = np.empty((n_pop, n_var))
    for j in range(n_var):
        lo, hi = lower_bounds[j], upper_bounds[j]
        if not isinf(lo) and not isinf(hi):
            pop_x[:, j] = rng.uniform(lo, hi, size=n_pop)
        elif isinf(lo) and isinf(hi):
            pop_x[:, j] = rng.normal(size=n_pop)
        elif isinf(hi):
            pop_x[:, j] = lo + rng.exponential(size=n_pop)
        else:  # isinf(lo)
            pop_x[:, j] = hi - rng.exponential(size=n_pop)

    pop_cost = np.apply_along_axis(F, 1, pop_x)

    best_idx = int(np.argmin(pop_cost))
    best_x = pop_x[best_idx].copy()
    best_cost = pop_cost[best_idx]
    best_grad = np.zeros(n_var)
    prev_mean = np.inf

    for it in range(1, max_it + 1):
        mean_cost = pop_cost.mean()
        if abs(prev_mean - mean_cost) <= tol_cost and np.linalg.norm(best_grad) <= tol_grad:
            break
        prev_mean = mean_cost

        for i in range(n_pop):
            others = [j for j in range(n_pop) if j != i]
            a, b, c = rng.choice(others, 3, replace=False)
            beta = rng.uniform(beta_min, beta_max, size=n_var)
            y = pop_x[a] + beta * (pop_x[b] - pop_x[c])
            y = np.minimum(np.maximum(y, lower_bounds), upper_bounds)

            j0 = rng.integers(n_var)
            mask = rng.random(n_var) < pCR
            mask[j0] = True
            z = np.where(mask, y, pop_x[i])
            z_cost = F(z)

            if z_cost < pop_cost[i]:
                pop_x[i], pop_cost[i] = z, z_cost
                if z_cost < best_cost:
                    best_x, best_cost = z.copy(), z_cost

            g = dF(pop_x[i])
            H = ddF(pop_x[i])
            try:
                direction = -np.linalg.solve(H, g)
            except np.linalg.LinAlgError:
                continue

            alphas = np.linspace(-1, 1, newton_LS_size)
            candidates = pop_x[i] + np.outer(alphas, direction)
            candidates = np.clip(candidates, lower_bounds, upper_bounds)
            costs_line = np.apply_along_axis(F, 1, candidates)

            idx_min = int(np.argmin(costs_line))
            z_new = candidates[idx_min]
            z_cost = costs_line[idx_min]

            if z_cost < pop_cost[i]:
                pop_x[i], pop_cost[i] = z_new, z_cost
                if z_cost < best_cost:
                    best_x, best_cost = z_new.copy(), z_cost
                    best_grad = g

    return best_x, it


In [ ]:

# Target parameters
a_true = np.array([0.8, 0.4])
b_true = np.array([1.0, 3.0])

# objective using parametrisation a=exp(a_params), b=exp(cumsum(b_params))
def objective(p):
    a_params = p[:2]
    b_params = p[2:]
    a = np.exp(a_params)
    b = np.exp(np.cumsum(b_params))
    diff = np.concatenate([a - a_true, b - b_true])
    return 0.5 * np.sum(diff * diff)

def grad(p):
    a_params = p[:2]
    b_params = p[2:]
    a = np.exp(a_params)
    b1 = np.exp(b_params[0])
    b2 = np.exp(b_params[0] + b_params[1])
    g = np.zeros_like(p)
    g[0] = (a[0] - a_true[0]) * a[0]
    g[1] = (a[1] - a_true[1]) * a[1]
    g[2] = (b1 - b_true[0]) * b1 + (b2 - b_true[1]) * b2
    g[3] = (b2 - b_true[1]) * b2
    return g

def hess(p):
    a_params = p[:2]
    b_params = p[2:]
    a = np.exp(a_params)
    b1 = np.exp(b_params[0])
    b2 = np.exp(b_params[0] + b_params[1])
    H = np.zeros((4,4))
    H[0,0] = (2*a[0] - a_true[0]) * a[0]
    H[1,1] = (2*a[1] - a_true[1]) * a[1]
    H_bb1 = (2*b1 - b_true[0]) * b1
    H_bb2 = (2*b2 - b_true[1]) * b2
    H[2,2] = H_bb1 + H_bb2
    H[2,3] = H_bb2
    H[3,2] = H_bb2
    H[3,3] = H_bb2
    return H

lower = np.array([-np.inf, -np.inf, -np.inf, 0.0])
upper = np.array([np.inf, np.inf, np.inf, np.inf])

best, it = fmin_DENewton(
    objective, grad, hess,
    lower, upper,
    beta_min=0.5, beta_max=1.0, pCR=0.9,
    n_pop=20, tol_cost=1e-8, tol_grad=1e-8, max_it=100,
)
print('best params:', best)
print('iterations:', it)
print('recovered a:', np.exp(best[:2]))
print('recovered b:', np.exp(np.cumsum(best[2:])))
